In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
import cv2

In [4]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [6]:
CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks (batch_sizes={"Medium": 32}, epoch_sizes= {"Dev": 5,}, crop={"C0": False}, name_suffix="",)



In [7]:
CHEXPERT_BENCHMARKS.keys() 

dict_keys(['BCE_E5_B32_C0', 'WBCE_E5_B32_C0', 'CWBCE_E5_B32_C0'])

In [8]:
#CHESTXRAY14_BENCHMARKS.keys()

In [9]:
#[m.name for m in METRICS]

In [10]:
#CHEXPERT_COLUMNS

In [11]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('jupyterlab', 'http://dfa98af5f7a6.ngrok.io'), ('tensorboard', 'http://1e2d917af28e.ngrok.io')]


In [12]:
CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"].as_dict()

{'benchmark_name': 'Chexpert_BCE_E5_B32_C0',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 5,
 'optimizer': 'Adam',
 'learning_rate': 0.001,
 'loss': 'binary_crossentropy',
 'use_class_weights': False,
 'positive_weights': [9.878620147705078,
  20.72118377685547,
  8.361455917358398,
  2.115144968032837,
  24.747013092041016,
  4.289779186248779,
  15.004094123840332,
  37.66842269897461,
  6.685722351074219,
  11.302910804748535,
  2.591697931289673,
  62.54441833496094,
  24.893571853637695,
  1.927727460861206],
 'negative_weights': [1.1126301288604736,
  1.0507068634033203,
  1.1358426809310913,
  1.8967443704605103,
  1.042110562324524,
  1.303971767425537,
  1.071407675743103,
  1.0272713899612427,
  1.1758791208267212,
  1.097059965133667,
  1.6282598972320557,
  1.0162484645843506,
  1.0418522357940674,
  2.0779027938842773],
 'metrics': ['auc',
  'precision',
  'recall',
  'f2_score',
  'binary_accuracy',
  'accu

In [13]:
chexpert_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"], DenseNet121, CHEXPERT_COLUMNS)

In [14]:
#chexpert_exp.benchmark.as_dict()

In [15]:
#chexpert_exp.run()

In [16]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

In [17]:
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                                     name_suffix="_D256_DS0595_savetest4",
                                                     classes=columns_5,
                                                     # test_labels = "test.csv",
                                                     split_valid_size = 0.8,
                                                     split_test_size = 0.5,
                                                     train_labels = "train.csv",
                                                     # nan_replacement = -1, #float("NaN"),
                                                     batch_sizes = {"b": 32},
                                                     epoch_sizes = {"e": 2},
                                                     dim=(256, 256),
                                                     crop = {"C0": False},
                                                     split_seed = 6122156,
                                                     augmentation=None)#"affine")

Chestxray_BCE_E2_B32_C0_N5_D256_DS0595_savetest4 could not be created
Chestxray_WBCE_E2_B32_C0_N5_D256_DS0595_savetest4 could not be created
Chestxray_CWBCE_E2_B32_C0_N5_D256_DS0595_savetest4 could not be created


In [18]:
chexpert_benchmarks.keys()

dict_keys(['BCE_E2_B32_C0_N5_D256_DS0595_savetest4', 'WBCE_E2_B32_C0_N5_D256_DS0595_savetest4', 'CWBCE_E2_B32_C0_N5_D256_DS0595_savetest4'])

In [19]:
chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_test"].as_dict()

KeyError: 'BCE_E1_B32_C0_N5_D256_DS0595_test'

In [21]:
chexpert_exp = simple_architecture_experiment(chexpert_benchmarks["BCE_E2_B32_C0_N5_D256_DS0595_savetest4"], DenseNet121, columns_5)

In [ ]:
#chexpert_exp.benchmark.as_dict()

In [ ]:
#[(m, m.name) for m in chexpert_exp.benchmark.metrics]

In [ ]:
chexpert_exp.train()

Epoch 1/2
700/700 [==============================] - ETA: 0s - loss: 0.4236 - auc: 0.6503 - precision: 0.5892 - recall: 0.2933 - f2_score: 0.3250 - binary_accuracy: 0.8252 - accuracy_cardiomegaly: 0.8777 - accuracy_edema: 0.7603 - accuracy_consolidation: 0.9336 - accuracy_atelectasis: 0.8484 - accuracy_pleural_effusion: 0.7060 - auc_cardiomegaly: 0.6276 - auc_edema: 0.7271 - auc_consolidation: 0.6026 - auc_atelectasis: 0.5378 - auc_pleural_effusion: 0.7566 - precision_cardiomegaly: 0.3270 - precision_edema: 0.4793 - precision_consolidation: 0.0400 - precision_atelectasis: 0.1795 - precision_pleural_effusion: 0.6360 - recall_cardiomegaly: 0.0279 - recall_edema: 0.1815 - recall_consolidation: 0.0052 - recall_atelectasis: 0.0146 - recall_pleural_effusion: 0.5858 - f2_score_cardiomegaly: 0.0310 - f2_score_edema: 0.2060 - f2_score_consolidation: 0.0049 - f2_score_atelectasis: 0.0157 - f2_score_pleural_effusion: 0.5951 - learning rate: 0.0010INFO:tensorflow:Summary name epoch_learning rate i

In [23]:
chexpert_exp.train_result

In [ ]:
#chexpert_exp.save()

In [24]:
from datetime import datetime
model_filename = chexpert_exp.model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_filename_tf = model_filename + ".tf"
model_filename_h5 = model_filename + ".h5"
model_filename

'DenseNet121_Chexpert_BCE_E2_B32_C0_N5_D256_DS0595_savetest4_20200818-200703'

In [25]:
folderpath = Path(os.getcwd()) / 'models' / chexpert_exp.model_name
path = folderpath / model_filename
path

PosixPath('/srv/idp-radio-1/models/DenseNet121_Chexpert_BCE_E2_B32_C0_N5_D256_DS0595_savetest4/DenseNet121_Chexpert_BCE_E2_B32_C0_N5_D256_DS0595_savetest4_20200818-200703')

In [26]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(path, save_format="tf")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /srv/idp-radio-1/models/DenseNet121_Chexpert_BCE_E2_B32_C0_N5_D256_DS0595_savetest4/DenseNet121_Chexpert_BCE_E2_B32_C0_N5_D256_DS0595_savetest4_20200818-200703/assets


In [27]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(folderpath / model_filename_h5, save_format="h5")

In [28]:
!pwd

/srv/idp-radio-1


In [ ]:
folderpath / model_filename_h5

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
